In [ ]:
import sys
sys.path.append("../src")  # Sube un nivel y apunta a la carpeta src

from data_prep.eda import load_data, explore_data, plot_churn_distribution, plot_numerical_distributions, plot_correlation_matrix

df = load_data("../data/raw/customer_churn.csv")
explore_data(df)



In [ ]:
plot_churn_distribution(df, target_col="Churn")


In [ ]:
plot_numerical_distributions(df, exclude_cols=["customerID"])


In [ ]:

plot_correlation_matrix(df)